# Purpose: To run the full segmentation using the best scored method from 2_compare_auto_to_manual_threshold

Date Created: January 7, 2022

Dates Edited: January 26, 2022 - changed the ogd severity study to be the otsu data as the yen data did not run on all samples.

*Step 1: Import Necessary Packages*

In [8]:
# import major packages
import numpy as np
import matplotlib.pyplot as plt
import skimage
import PIL as Image
import os
import pandas as pd

# import specific package functions
from skimage.filters import threshold_otsu
from skimage import morphology
from scipy import ndimage
from skimage.measure import label
from skimage import io
from skimage import measure

__OGD Severity Study__

In [9]:
im_folder_location = '/Users/hhelmbre/Desktop/ogd_severity_undergrad/10_4_21_redownload/'

In [10]:
im_paths = []
files = []
for file in os.listdir(im_folder_location):
    if file.endswith(".tif"):
        file_name = os.path.join(im_folder_location, file)
        files.append(file)
        im_paths.append(file_name)

In [11]:
files

['4-50-6_40x_thalamus_1.tif',
 '4-59_1_40x_thalamus_2.tif',
 '4-56-7_40x_thalamus_3.tif',
 '4-50-5_40x_thalamus_5.tif',
 '4-56-10_40x_thalamus_2.tif',
 '4-59_2_40x_cortex_6.tif',
 '4-50-8_40x_hippocampus_4.tif',
 '4-50-5_40x_cortex_2.tif',
 '4-50-8_40x_thalamus_5.tif',
 '4-50-8_40x_thalamus_4.tif',
 '4-50-5_40x_cortex_3.tif',
 '4-50-8_40x_hippocampus_5.tif',
 '4-56-10_40x_thalamus_3.tif',
 '4-50-5_40x_thalamus_4.tif',
 '4-56-7_40x_thalamus_2.tif',
 '4-59_1_40x_thalamus_3.tif',
 '4-50-6_40x_thalamus_2.tif',
 '4-56-4_40x_thalamus_4.tif',
 '4-50-1_40x_cortex_5.tif',
 '4-56-9_40x_thalamus_4.tif',
 '4-50-15_40x_thalamus_4.tif',
 '4-56-4_40x_cortex_5.tif',
 '4-59_1_40x_thalamus_1.tif',
 '4-50-12_40x_cortex_4.tif',
 '4-50-15_40x_hippocampus_4.tif',
 '4-59_2_40x_thalamus_5.tif',
 '4-56-10_40x_thalamus_1.tif',
 '4-59_2_40x_cortex_5.tif',
 '4-50-5_40x_cortex_1.tif',
 '4-59_2_40x_cortex_4.tif',
 '4-59_2_40x_thalamus_4.tif',
 '4-50-15_40x_hippocampus_5.tif',
 '4-50-12_40x_cortex_5.tif',
 '4-56-7_4

In [12]:
properties_list = ('area', 'bbox_area', 'centroid', 'convex_area', 
                   'eccentricity', 'equivalent_diameter', 'euler_number', 
                   'extent', 'filled_area', 'major_axis_length', 
                   'minor_axis_length', 'orientation', 'perimeter', 'solidity')

In [13]:
source_dir = '/Users/hhelmbre/Desktop/microfiber/ogd_severity_segmentations/'

In [14]:
j = 0
for image in im_paths:
    
    short_im_name = image.rsplit('/', 1)
    short_im_name = short_im_name[1]
    
    im = io.imread(image)
    microglia_im = im[:,:,1]
    #otsu threshold
    thresh_otsu = skimage.filters.threshold_otsu(microglia_im)
    binary_otsu = microglia_im > thresh_otsu
    new_binary_otsu = morphology.remove_small_objects(binary_otsu, min_size=71)
    new_binary_otsu = ndimage.binary_fill_holes(new_binary_otsu)
    label_image = measure.label(new_binary_otsu)
    props = measure.regionprops_table(label_image, properties=(properties_list))
    
    np.save(str(source_dir + short_im_name[:-4] + '_otsu_thresh'), new_binary_otsu)

    if j == 0:
        df = pd.DataFrame(props)
        df['filename'] = image
    else:
        df2 = pd.DataFrame(props)
        df2['filename'] = image
        df = df.append(df2)

    j = 1
    
    

In [15]:
df['circularity'] = 4*np.pi*df.area/df.perimeter**2

In [16]:
df['aspect_ratio'] = df.major_axis_length/df.minor_axis_length

In [17]:
df

,area,bbox_area,centroid-0,centroid-1,convex_area,eccentricity,equivalent_diameter,euler_number,extent,filled_area,major_axis_length,minor_axis_length,orientation,perimeter,solidity,filename,circularity,aspect_ratio
0,187,308,5.449198,499.604278,203,0.814776,15.430353,1,0.607143,187,20.905669,12.120597,1.080724,61.077164,0.921182,/Users/hhelmbre/Desktop/ogd_severity_undergrad...,0.629932,1.724805
1,236,483,28.474576,480.105932,302,0.829253,17.334490,1,0.488613,236,24.320267,13.591965,-0.744975,76.526912,0.781457,/Users/hhelmbre/Desktop/ogd_severity_undergrad...,0.506400,1.789312
2,325,644,71.713846,340.523077,465,0.714540,20.342145,1,0.504658,325,30.657592,21.447887,-0.370980,139.539105,0.698925,/Users/hhelmbre/Desktop/ogd_severity_undergrad...,0.209750,1.429399
3,84,168,65.988095,365.297619,117,0.491810,10.341766,1,0.500000,84,12.501461,10.885056,-0.335525,45.834524,0.717949,/Users/hhelmbre/Desktop/ogd_severity_undergrad...,0.502463,1.148498
4,163,308,70.926380,232.000000,231,0.903842,14.406181,1,0.529221,163,25.356380,10.849124,-1.234703,65.941125,0.705628,/Users/hhelmbre/Desktop/ogd_severity_undergrad...,0.471069,2.337182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,226,324,170.969027,382.097345,244,0.418603,16.963258,1,0.697531,226,17.889276,16.246490,0.850648,57.734019,0.926230,/Users/hhelmbre/Desktop/ogd_severity_undergrad...,0.852030,1.101116
4,232,483,184.068966,320.961207,361,0.266652,17.186959,1,0.480331,232,23.330253,22.485532,0.051090,125.225397,0.642659,/Users/hhelmbre/Desktop/ogd_severity_undergrad...,0.185914,1.037567
5,332,440,259.852410,313.394578,347,0.336332,20.560047,1,0.754545,332,21.203766,19.968511,-1.322205,67.112698,0.956772,/Users/hhelmbre/Desktop/ogd_severity_undergrad...,0.926272,1.061860
6,165,240,258.672727,499.903030,177,0.598762,14.494293,1,0.687500,165,16.367323,13.109026,0.533650,51.284271,0.932203,/Users/hhelmbre/Desktop/ogd_severity_undergrad...,0.788362,1.248554


In [18]:
df.to_csv('/Users/hhelmbre/Desktop/microfiber/ogd_severity_study_features_otsu.csv' )

In [19]:
%load_ext watermark

%watermark -v -m -p numpy,pandas,scipy,skimage,matplotlib,wget

%watermark -u -n -t -z

Python implementation: CPython
Python version       : 3.7.4
IPython version      : 7.8.0

numpy     : 1.21.5
pandas    : 1.3.5
scipy     : 1.3.1
skimage   : 0.17.2
matplotlib: 3.1.1
wget      : 3.2

Compiler    : Clang 4.0.1 (tags/RELEASE_401/final)
OS          : Darwin
Release     : 20.6.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit

Last updated: Wed Jan 26 2022 14:39:49PST

